In [12]:
#Locating city incinerators from city directories_Case study: NC

In [2]:
#Checking binarization (if images are binary)

In [6]:
import os
import cv2
import numpy as np
from glob import glob

def is_binary(img):
    # Unique pixel values in the image
    unique_vals = np.unique(img)
    return np.array_equal(unique_vals, [0, 255]) or np.array_equal(unique_vals, [0]) or np.array_equal(unique_vals, [255])

base_dir = 'path_to_folder'
img_files = glob(os.path.join(base_dir, '*.png')) + glob(os.path.join(base_dir, '*.jpg'))

for fname in img_files:
    img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Cannot open {fname}")
        continue

    unique = np.unique(img)
    if is_binary(img):
        print(f"{os.path.basename(fname)} is BINARY. (Unique values: {unique})")
    else:
        print(f"{os.path.basename(fname)} is NOT binary! (Unique values: {unique})")



Image1.jpg is NOT binary! (Unique values: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 

In [4]:
#Conversion into binary images

In [7]:
import cv2
import os
from glob import glob

# Set your directory path
src_folder ='path_to_folder'
out_folder = os.path.join(src_folder, 'binary1')

# Create output directory
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

# Collect image files
img_files = glob(os.path.join(src_folder, '*.png')) + glob(os.path.join(src_folder, '*.jpg'))

for fname in img_files:
    # Read in grayscale mode
    img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Could not read {fname}, skipping.")
        continue

    # Use Otsu's method for robust thresholding
    _, bin_img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # You can use fixed threshold instead:
    # _, bin_img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # Save
    out_name = os.path.join(out_folder, os.path.basename(fname))
    cv2.imwrite(out_name, bin_img)
    print(f"Saved binary image: {out_name}")


Saved binary image: C:\Users\anseb\Downloads\FinalProject\City directories\column1\binary1\Image1.jpg
Saved binary image: C:\Users\anseb\Downloads\FinalProject\City directories\column1\binary1\Image10.jpg
Saved binary image: C:\Users\anseb\Downloads\FinalProject\City directories\column1\binary1\Image11.jpg
Saved binary image: C:\Users\anseb\Downloads\FinalProject\City directories\column1\binary1\Image12.jpg
Saved binary image: C:\Users\anseb\Downloads\FinalProject\City directories\column1\binary1\Image13.jpg
Saved binary image: C:\Users\anseb\Downloads\FinalProject\City directories\column1\binary1\Image14.jpg
Saved binary image: C:\Users\anseb\Downloads\FinalProject\City directories\column1\binary1\Image15.jpg
Saved binary image: C:\Users\anseb\Downloads\FinalProject\City directories\column1\binary1\Image16.jpg
Saved binary image: C:\Users\anseb\Downloads\FinalProject\City directories\column1\binary1\Image17.jpg
Saved binary image: C:\Users\anseb\Downloads\FinalProject\City directories

In [ ]:
#Calculating image size and adapting padding to that to provide a single line entry

In [9]:
import os
import cv2
import numpy as np
from glob import glob

base_dir ='path_to_folder\binary1'
entry_dir = os.path.join(base_dir, 'entry_lines1')
os.makedirs(entry_dir, exist_ok=True)

def estimate_line_height(img):
    # binary image assumed: text = 0, background = 255
    h, w = img.shape[:2]
    black_counts = np.array([np.sum(img[i, :] == 0) for i in range(h)])

    # gap rows ≈ mostly white
    gap_frac = 0.05
    gap_thresh = gap_frac * w
    gap_rows = [i for i, val in enumerate(black_counts) if val <= gap_thresh]
    cuts = [0] + gap_rows + [h]

    line_heights = []
    for s, e in zip(cuts[:-1], cuts[1:]):
        if e - s > 3 and np.sum(black_counts[s:e]) > 0:
            line_heights.append(e - s)
    if not line_heights:
        return None
    return np.median(line_heights)

def crop_lines(img, padding):
    h, w = img.shape[:2]
    black_counts = np.array([np.sum(img[i, :] == 0) for i in range(h)])
    gap_frac = 0.05
    gap_thresh = gap_frac * w
    gap_rows = [i for i, val in enumerate(black_counts) if val <= gap_thresh]
    cuts = [0] + gap_rows + [h]
    lines = []
    for s, e in zip(cuts[:-1], cuts[1:]):
        if e - s < 3:
            continue
        region = img[s:e, :]
        if np.sum(region == 0) / region.size < 0.05:
            continue
        top = max(s - padding, 0)
        bot = min(e + padding, h)
        lines.append(img[top:bot, :])
    return lines

img_files = glob(os.path.join(base_dir, '*.png')) + glob(os.path.join(base_dir, '*.jpg'))

for fname in img_files:
    img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    # 1) estimate line height for this image
    line_h = estimate_line_height(img)
    if line_h is None:
        print(f"No lines detected in {fname}")
        continue

    # 2) define padding as, say, 20% of line height (tune as needed)
    padding = int(0.15 * line_h)
    print(f"{os.path.basename(fname)}: median line height={line_h}, padding={padding}")

    # 3) crop individual lines using this padding
    lines = crop_lines(img, padding)

    # 4) save outputs
    base = os.path.splitext(os.path.basename(fname))[0]
    for i, line_img in enumerate(lines, start=1):
        outname = os.path.join(entry_dir, f"{base}_line_{i}.png")
        cv2.imwrite(outname, line_img)


Image1.jpg: median line height=43.0, padding=6
Image10.jpg: median line height=51.0, padding=7
Image11.jpg: median line height=25.0, padding=3
Image12.jpg: median line height=18.0, padding=2
Image13.jpg: median line height=17.0, padding=2
Image14.jpg: median line height=16.0, padding=2
Image15.jpg: median line height=25.0, padding=3
Image16.jpg: median line height=26.0, padding=3
Image17.jpg: median line height=23.0, padding=3
Image18.jpg: median line height=21.0, padding=3
Image19.jpg: median line height=31.5, padding=4
Image2.jpg: median line height=21.5, padding=3
Image20.jpg: median line height=31.5, padding=4
Image21.jpg: median line height=21.0, padding=3
Image22.jpg: median line height=29.5, padding=4
Image23.jpg: median line height=30.0, padding=4
Image24.jpg: median line height=32.0, padding=4
Image25.jpg: median line height=20.0, padding=3
Image26.jpg: median line height=44.5, padding=6
Image27.jpg: median line height=34.5, padding=5
Image28.jpg: median line height=42.5, padd

In [ ]:
#Identifying incinerator from each line 

In [10]:
import os
import cv2
import pytesseract
from glob import glob

# 1. Configure paths
line_dir = r"C:\Users\anseb\Downloads\FinalProject\City directories\column1\binary1\entry_lines1"
# Folder where your per-line crops were saved in the previous step

# 2. Point pytesseract to the Tesseract executable (update this path!)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# 3. Helper: check if 'incinerator' appears in OCR text
def contains_incinerator(text: str) -> bool:
    return "incinerator" in text.lower()

# 4. Loop over all line images and OCR them
results = []  # will store (filename, text) for matches

# You can adapt the pattern if your lines are elsewhere
img_files = glob(os.path.join(line_dir, "*.png")) + glob(os.path.join(line_dir, "*.jpg"))

for fname in sorted(img_files):
    img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Could not read {fname}, skipping.")
        continue

    # Optional small preprocessing to help OCR on binary text lines
    # (comment out if not needed)
    # img = cv2.medianBlur(img, 3)

    text = pytesseract.image_to_string(img, lang="eng")  # add config=... if needed
    if contains_incinerator(text):
        print(f"FOUND 'incinerator' in {os.path.basename(fname)}:")
        print(text.strip())
        print("-" * 60)
        results.append((fname, text))

# 5. Optionally, write matches to a text file
out_txt = os.path.join(line_dir, "incinerator_hits1.txt")
with open(out_txt, "w", encoding="utf-8") as f:
    for fname, text in results:
        f.write(f"=== {os.path.basename(fname)} ===\n")
        f.write(text.strip() + "\n\n")

print(f"\nDone. {len(results)} lines contained 'incinerator'. Results saved to:\n{out_txt}")


FOUND 'incinerator' in Image18_line_51.png:
City Hall 120-124 Morris. Tel | INCINERATOR, E Bernice Parrish
------------------------------------------------------------
FOUND 'incinerator' in Image19_line_16.png:
12 Lipscomb Haywood (c) 7 wv
WABoykins’ Clars (¢) clo clars | FERRELL ROAD (Durham County)
15 Toomer Nora {¢) —Cantinuation of E Trintty av
21 Barnes Edgar D (c) © | beyond city limits
White Thos Q (c) @ \ SCity Master Incinerator

18154Morris Jas T (c} ©
10164Woodward Mary J {(c) & 5
16161, 4Coltege Heights Beau

‘ Shop (c)
161748mith Wm (ec)
------------------------------------------------------------
FOUND 'incinerator' in Image22_line_14.png:
Eleven Corcoran Bidg, 111 N Corcor- | .- Master Incinerator Ferrell rd bey city
------------------------------------------------------------
FOUND 'incinerator' in Image26_line_20.png:
Acts
Health Department, Dr Jesse H Epperson supt 300 Court House
Incinerator, Marvin A Gunter supt Carroll nr Cobb, 1201 Cates av,
---------------------